# Cats vs Dogs -- with help from a pre-trained image classification model!

<img src="https://raw.githubusercontent.com/benjum/UCLA-24W-DH150/main/Data/cat-dog-data/test/cat/cat.1500.jpg" width="250px"/> <img src="https://raw.githubusercontent.com/benjum/UCLA-24W-DH150/main/Data/cat-dog-data/test/dog/dog.1500.jpg" width="250px"/>

#### We're going to use TensorFlow and Convolutional Neural Networks to identify whether a picture is a dog or a cat.

Side-note:  I think the dataset is biased against cats, because most of the cats I've looked at in this dataset look crazy, a bit ugly, or like they totally have it out for the dogs!

This dataset is a subset of the data that can be obtained at [Kaggle's Dogs Vs Cats page](https://www.kaggle.com/c/dogs-vs-cats/data).

The code below was motivated by Chollet's Deep Learning with Python book, in which you will find many, many more interesting details and tidbits about doing deep learning.

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

Special note:  The below can take a long time to run if you only run it using your computer's CPU.  It is highly recommended that you get TensorFlow running on your GPUs if you want to reproduce the cell execution below.  I (B. Winjum) ran this on my Mac M1 and am doing this with TensorFlow 2.13 (along with tensorflow-metal) so that I can run on a GPU.  If you need help running this notebook and/or getting tensorflow running on your local GPU, please reach out for assistance as needed.

In [ ]:
tf.__version__

In [ ]:
tf.config.list_physical_devices()

In [ ]:
# tf.get_logger().setLevel('ERROR')

## Getting our data

Our data images are in folders that are already split up into training, test, and validation sets.  Furthermore, they are aleady split up into cat and dog folders.

The following image_dataset_from_directory gets class labels based on whether the images are retrieved from these "cat" or "dog" folders.

In [ ]:
import os

In [ ]:
os.path.join(os.getcwd(), "..", "..", "Data", "cat-dog-data", "train")

In [ ]:
train_dataset = image_dataset_from_directory(
    os.path.join(os.getcwd(), "..", "..", "Data", "cat-dog-data", "train"),
    image_size=(180, 180),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    os.path.join(os.getcwd(), "..", "..", "Data", "cat-dog-data", "validation"),
    image_size=(180, 180),
    batch_size=32)
test_dataset = image_dataset_from_directory(
    os.path.join(os.getcwd(), "..", "..", "Data", "cat-dog-data", "test"),
    image_size=(180, 180),
    batch_size=32)

In [ ]:
train_ds = train_dataset.unbatch()
a = list(train_ds)

In [ ]:
len(a)

In [ ]:
a[2][0].shape

In [ ]:
a[2][1]

In [ ]:
a[2][0][0][2]

In [ ]:
a[2][1]

In [ ]:
b = 2
print("The label for image",b,"is",a[b][1].numpy())
print("The picture for image",b,"is")
plt.imshow(a[b][0].numpy().astype('int32'));

# Using pre-trained model

We will try using a pre-trained model available through Keras Applications.  "Keras Applications are deep learning models that are made available alongside pre-trained weights. These models can be used for prediction, feature extraction, and fine-tuning."
* https://keras.io/api/applications/

## Feature extraction without data augmentation

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(180, 180, 3))

In [ ]:
conv_base.summary()

In [ ]:
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)
  
train_features, train_labels =  get_features_and_labels(train_dataset)
val_features, val_labels =  get_features_and_labels(validation_dataset)
test_features, test_labels =  get_features_and_labels(test_dataset)

In [ ]:
train_features.shape

In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
  
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction.keras",
        save_best_only=True,
        monitor="val_loss",
        save_weights_only=True)
]

history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
model.load_weights("feature_extraction.keras")

In [ ]:
model.summary()

In [ ]:
test_loss, test_acc = model.evaluate(test_features, test_labels)

In [ ]:
test_acc

## Feature extraction with data augmentation

In [ ]:
conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)

In [ ]:
conv_base.trainable = True
len(conv_base.trainable_weights)

In [ ]:
conv_base.trainable = False
len(conv_base.trainable_weights)

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss",
        save_weights_only=True
    )
]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
# this loads the best weights into the model
model.load_weights("feature_extraction_with_data_augmentation.keras")

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

In [ ]:
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
model.summary()

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
conv_base.summary()

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss",
        save_weights_only=True
    )
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
model.load_weights("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")